# Segment clouds and cloud shadows in Landsat images (L2A)
This notebook shows an example on how to use [ukis-csmask](https://github.com/dlr-eoc/ukis-csmask) to segment clouds and cloud shadows in Level-2A images from Landsat-9, Landsat-8, Landsat-7 and Landsat-5 satellites. Images are acquired from [Planetary Computer](https://planetarycomputer.microsoft.com) and are preprocessed.

> NOTE: to run this notebook, we first need to install some additional dependencies for work with Planetary Computer
```shell
$ pip install planetary_computer rioxarray pystac-client odc-stac tqdm
```

In [ ]:
import numpy as np
import planetary_computer as pc
import rioxarray

from odc.stac import load
from pathlib import Path
from pystac_client import Client
from tqdm import tqdm
from ukis_csmask.mask import CSmask
from xarray import DataArray

In [ ]:
# user settings
stac_api_endpoint = "https://planetarycomputer.microsoft.com/api/stac/v1"
collections = ["landsat-c2-l2"]
ids = [
    "LC09_L2SP_192027_20240403_02_T2",
    "LC08_L2SP_192027_20240801_02_T1",
    "LE07_L2SP_192027_20000722_02_T1",
    "LT05_L2SP_192027_20100726_02_T1",
]
bbox = [11.540, 47.463, 12.117, 47.872]
product_level = "l2a"
band_order = ["blue", "green", "red", "nir", "swir16", "swir22"]
providers = ["CUDAExecutionProvider"]
out_dir = "ukis-csmask/examples"

In [ ]:
# search catalog by scene id
catalog = Client.open(stac_api_endpoint)
search = catalog.search(collections=collections, ids=ids)
items = [item for item in search.items()]
items_cnt = len(items)
print(f"Search returned {items_cnt} item(s)")

In [ ]:
for item in tqdm(items, total=items_cnt, desc="Predict images"):
    # near infrared band has different alias in landsat collections
    bands = [b.replace("nir", "nir08") for b in band_order]

    # load and preprocess image
    arr = (
        load(
            items=[item],
            bands=bands,
            resolution=30,
            bbox=bbox,
            dtype="float32",
            patch_url=pc.sign,
        )
        .to_dataarray()
        .squeeze()
        .drop_vars("time")
    )
    arr = arr.rename({"variable": "band"})

    # use band-specific rescale factors to convert DN to reflectance
    for idx, band_name in enumerate(bands):
        band_info = item.assets[band_name].extra_fields["raster:bands"][0]
        arr[idx, :, :] = arr.sel(band=str(band_name)).astype(np.float32) * band_info["scale"]
        arr[idx, :, :] += band_info["offset"]
        arr[idx, :, :] = arr[idx, :, :].clip(min=0.0, max=1.0)

    # compute cloud and cloud shadow mask
    csmask = CSmask(
        img=np.moveaxis(arr.to_numpy(), 0, -1),
        band_order=band_order,
        product_level=product_level,
        nodata_value=0,
        invalid_buffer=4,
        intra_op_num_threads=0,
        inter_op_num_threads=0,
        providers=providers,
        batch_size=1,
    )

    # write image, csm and valid mask to file
    arr.rio.to_raster(raster_path=Path(out_dir) / Path(f"{item.id}.tif"), driver="COG")
    DataArray(np.squeeze(np.moveaxis(csmask.csm, -1, 0)), coords=arr.sel(band=arr["band"][0]).coords).rio.to_raster(
        raster_path=Path(out_dir) / Path(f"{item.id}_csm.tif"), driver="COG"
    )
    DataArray(np.squeeze(np.moveaxis(csmask.valid, -1, 0)), coords=arr.sel(band=arr["band"][0]).coords).rio.to_raster(
        raster_path=Path(out_dir) / Path(f"{item.id}_valid.tif"), driver="COG"
    )